In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
import scipy.cluster.hierarchy as sch
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import requests
import datetime
import time
import warnings
import ta
warnings.filterwarnings('ignore')

In [7]:
stocklist = si.tickers_sp500()
start_date = datetime.datetime.now() - datetime.timedelta(days=(365*3) + 30)
end_date = datetime.date.today()

In [10]:
def get_data(ticker,start_date,end_date):
    data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
    data['Ticker'] = ticker
    return data


def technical_analysis(sec):
    '''
    Contains all the indicators
    sec: security dataframe - should contain Close, Open, Low, High, and Volume of the desired security
    returns all the technical indicators
    
    '''
    #Momentum Indicators
    stoc_oscc_signal = ta.momentum.stoch_signal(high = sec.High, low = sec.Low,close = sec.Close, n=14, d_n=3, fillna=False)
    tsi = ta.momentum.tsi(close = sec.Close, r=25, s=13, fillna=False)
    awesome_oscillator = ta.momentum.AwesomeOscillatorIndicator(high = sec.High, low = sec.Low, s = 5, len = 34, fillna = False).ao()
    kama_indicator = ta.momentum.KAMAIndicator(close = sec.Close, n = 10, pow1 = 2, pow2 = 30, fillna = False).kama()
    roc_indicator = ta.momentum.ROCIndicator(close = sec.Close, n = 12, fillna = False).roc()
    rsi14 = ta.momentum.rsi(close = sec.Close, n=14, fillna=False)
    ultimate_osc = ta.momentum.UltimateOscillator(high = sec.High, low = sec.Low,close =  sec.Close, s = 7, m = 14, len = 28, ws = 4.0, wm = 2.0, wl = 1.0, fillna = False).uo()
    williamsR = ta.momentum.WilliamsRIndicator(high = sec.High, low = sec.Low,close =  sec.Close, lbp = 14, fillna = False).wr()
    
    #Volume Indicators
    adi = ta.volume.AccDistIndexIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, fillna = False).acc_dist_index()
    chaikin_money_flow = ta.volume.ChaikinMoneyFlowIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, n= 20, fillna = False).chaikin_money_flow()
    ease_of_movement = ta.volume.EaseOfMovementIndicator(high = sec.High, low = sec.Low, volume = sec.Volume, n= 14, fillna = False)
    force_index_indicator = ta.volume.ForceIndexIndicator(close = sec.Close, volume = sec.Volume, n = 13, fillna = False).force_index()
    mfi = ta.volume.MFIIndicator(high = sec.High, low = sec.Low, close = sec.Close, volume= sec.Volume, n = 14, fillna = False).money_flow_index()
    nvi = ta.volume.NegativeVolumeIndexIndicator(close = sec.Close, volume = sec.Volume, fillna= False).negative_volume_index()
    obv = ta.volume.OnBalanceVolumeIndicator(close = sec.Close, volume = sec.Volume, fillna = False).on_balance_volume()
    priceTrend = ta.volume.VolumePriceTrendIndicator(close = sec.Close, volume = sec.Volume, fillna = False).volume_price_trend()
    vwap = ta.volume.VolumeWeightedAveragePrice(high = sec.High, low = sec.Low, close = sec.Close, volume = sec.Volume, n= 14, fillna= False).volume_weighted_average_price()
    
    #Volatility Indicators
    atr = ta.volatility.AverageTrueRange(high = sec.High, low = sec.Low, close = sec.Close, n= 14, fillna= False).average_true_range()
    
    #Trend Indicators
    adx = ta.trend.ADXIndicator(high = sec.High, low = sec.Low, close = sec.Close, n= 14, fillna = False).adx()
    aroon = ta.trend.AroonIndicator(close = sec.Close, n = 25, fillna = False).aroon_indicator()
    cci = ta.trend.CCIIndicator(high = sec.High, low = sec.Low, close = sec.Close, n = 20, c= 0.015, fillna= False).cci()
    dpo = ta.trend.DPOIndicator(close = sec.Close, n = 20, fillna = False).dpo()
    ema = ta.trend.EMAIndicator(close = sec.Close, n = 14, fillna = False).ema_indicator()
    kst = ta.trend.KSTIndicator(close= sec.Close, r1 = 10, r2 = 15, r3= 20, r4 = 30, n1= 10, n2= 10, n3 = 10, n4 = 15, nsig = 9, fillna= False).kst()
    macd = ta.trend.MACD(close = sec.Close, n_slow = 26, n_fast = 12, n_sign= 9, fillna = False).macd()
    
    tech = pd.DataFrame({'stoc_oscc_signal':stoc_oscc_signal,'tsi':tsi,'awesome_oscillator':awesome_oscillator,'kama_indicator':kama_indicator,
    'roc_indicator':roc_indicator,'rsi14':rsi14,'ultimate_osc':ultimate_osc,'williamsR':williamsR,
    'adi':adi,"chaikin_money_flow":chaikin_money_flow,'ease_of_movement':ease_of_movement,'force_index_indicator':force_index_indicator,
    'mfi':mfi,'nvi':nvi,'obv':obv,'priceTrend':priceTrend,'vwap':vwap,'atr':atr,
    'adx':adx,'aroon':aroon,'cci':cci,'dpo':dpo,'ema':ema, 'kst':kst,'macd':macd})
    
    return tech


def merge_returns_and_technicals(price_df,technicals_df):
    ticker = price_df['Ticker'][0]
    price_df.drop(columns = ['Ticker'],inplace = True)
    returns = price_df.pct_change(1).rename(columns = {'Adj Close': 'Returns'})
    merged_df = technicals_df.join(returns['Returns'].shift(1)) #Lagged Returns
    merged_df['Ticker'] = ticker
    return merged_df


def master_data(security_list,start_date,end_date):
    final_df = pd.DataFrame()
    count = 0
    for security in security_list:
        try:
            sec = get_data(security,start_date = start_date,end_date = end_date)
            tech = technical_analysis(sec)
            merged = merge_returns_and_technicals(sec,tech)
            final_df = pd.concat([final_df,merged],axis = 0).dropna()
        except:
            pass
        count+=1
        print(count)
    return final_df

In [11]:
master = master_data(stocklist,start_date=start_date,end_date=end_date)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
master

,stoc_oscc_signal,tsi,awesome_oscillator,kama_indicator,roc_indicator,rsi14,ultimate_osc,williamsR,adi,chaikin_money_flow,...,atr,adx,aroon,cci,dpo,ema,kst,macd,Returns,Ticker
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-07,90.010534,25.827497,2.978501,64.014347,5.506961,71.153008,68.807550,-2.213291,9.656742e+05,0.129107,...,0.872473,31.280422,76.0,108.095884,0.545499,63.511878,21.659210,1.080766,0.006533,A
2017-09-08,91.403908,27.041132,3.036647,64.240404,3.271915,69.760310,63.216585,-13.983125,8.775143e+04,0.138377,...,0.870867,32.183206,76.0,112.786058,0.033000,63.712960,30.494717,1.104002,0.006645,A
2017-09-11,93.394588,29.689227,3.132941,64.732395,5.321013,74.619704,65.111793,-3.619821,1.221362e+06,0.156316,...,0.897948,33.409580,84.0,136.859364,-0.396000,64.032566,39.993588,1.196578,-0.001842,A
2017-09-12,91.839159,31.879345,3.383911,65.072346,5.520101,74.740013,60.665754,-6.879577,1.160125e+06,0.206327,...,0.871666,34.641836,88.0,136.373676,-0.501999,64.313557,48.666183,1.257866,0.016764,A
2017-09-13,88.474027,31.682963,3.492823,65.111992,4.038157,66.784726,59.478050,-24.078521,-7.311770e+05,0.183521,...,0.867261,34.906151,88.0,105.144495,-0.245001,64.463750,55.288269,1.235709,0.000454,A
2017-09-14,83.102875,31.879352,3.457265,65.168332,3.498269,68.040743,60.522064,-19.733276,-4.283608e+04,0.090565,...,0.856028,34.718265,88.0,89.665386,-0.391998,64.625916,60.760458,1.223412,-0.010584,A
2017-09-15,80.951628,32.459134,3.406118,65.223575,3.890372,69.490279,58.790645,-13.333319,7.224972e+05,0.222675,...,0.831312,34.770257,84.0,98.679634,-0.236999,64.803794,65.790047,1.222172,0.003667,A
2017-09-18,85.644502,33.117181,3.270236,65.331706,3.411581,70.115873,56.758512,-9.999899,1.139036e+06,0.214929,...,0.814790,34.965686,76.0,99.099524,0.341501,64.973955,71.359508,1.216846,0.004263,A
2017-09-19,89.172163,33.801853,3.102618,65.456685,2.240416,70.602706,58.533635,-9.150294,8.364099e+05,0.180465,...,0.788733,35.278526,96.0,103.289087,-0.220003,65.133428,72.487792,1.205984,0.001819,A


In [13]:
master.columns

Index(['stoc_oscc_signal', 'tsi', 'awesome_oscillator', 'kama_indicator',
       'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR', 'adi',
       'chaikin_money_flow', 'ease_of_movement', 'force_index_indicator',
       'mfi', 'nvi', 'obv', 'priceTrend', 'vwap', 'atr', 'adx', 'aroon', 'cci',
       'dpo', 'ema', 'kst', 'macd', 'Returns', 'Ticker'],
      dtype='object')

In [15]:
master = master[['Ticker','Returns','stoc_oscc_signal', 'tsi', 'awesome_oscillator', 'kama_indicator',
       'roc_indicator', 'rsi14', 'ultimate_osc', 'williamsR', 'adi',
       'chaikin_money_flow', 'ease_of_movement', 'force_index_indicator',
       'mfi', 'nvi', 'obv', 'priceTrend', 'vwap', 'atr', 'adx', 'aroon', 'cci',
       'dpo', 'ema', 'kst', 'macd']]

In [16]:
master

,Ticker,Returns,stoc_oscc_signal,tsi,awesome_oscillator,kama_indicator,roc_indicator,rsi14,ultimate_osc,williamsR,...,priceTrend,vwap,atr,adx,aroon,cci,dpo,ema,kst,macd
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-07,A,0.006533,90.010534,25.827497,2.978501,64.014347,5.506961,71.153008,68.807550,-2.213291,...,13515.011323,63.162808,0.872473,31.280422,76.0,108.095884,0.545499,63.511878,21.659210,1.080766
2017-09-08,A,0.006645,91.403908,27.041132,3.036647,64.240404,3.271915,69.760310,63.216585,-13.983125,...,4223.824475,63.593510,0.870867,32.183206,76.0,112.786058,0.033000,63.712960,30.494717,1.104002
2017-09-11,A,-0.001842,93.394588,29.689227,3.132941,64.732395,5.321013,74.619704,65.111793,-3.619821,...,26747.612240,63.966968,0.897948,33.409580,84.0,136.859364,-0.396000,64.032566,39.993588,1.196578
2017-09-12,A,0.016764,91.839159,31.879345,3.383911,65.072346,5.520101,74.740013,60.665754,-6.879577,...,30163.452509,64.216765,0.871666,34.641836,88.0,136.373676,-0.501999,64.313557,48.666183,1.257866
2017-09-13,A,0.000454,88.474027,31.682963,3.492823,65.111992,4.038157,66.784726,59.478050,-24.078521,...,-21127.054318,64.472918,0.867261,34.906151,88.0,105.144495,-0.245001,64.463750,55.288269,1.235709
2017-09-14,A,-0.010584,83.102875,31.879352,3.457265,65.168332,3.498269,68.040743,60.522064,-19.733276,...,-16186.497013,64.704053,0.856028,34.718265,88.0,89.665386,-0.391998,64.625916,60.760458,1.223412
2017-09-15,A,0.003667,80.951628,32.459134,3.406118,65.223575,3.890372,69.490279,58.790645,-13.333319,...,15219.512413,64.964261,0.831312,34.770257,84.0,98.679634,-0.236999,64.803794,65.790047,1.222172
2017-09-18,A,0.004263,85.644502,33.117181,3.270236,65.331706,3.411581,70.115873,56.758512,-9.999899,...,12629.807957,65.150243,0.814790,34.965686,76.0,99.099524,0.341501,64.973955,71.359508,1.216846
2017-09-19,A,0.001819,89.172163,33.801853,3.102618,65.456685,2.240416,70.602706,58.533635,-9.150294,...,4527.856696,65.320335,0.788733,35.278526,96.0,103.289087,-0.220003,65.133428,72.487792,1.205984


In [17]:
master.describe()

,Returns,stoc_oscc_signal,tsi,awesome_oscillator,kama_indicator,roc_indicator,rsi14,ultimate_osc,williamsR,adi,...,priceTrend,vwap,atr,adx,aroon,cci,dpo,ema,kst,macd
count,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,3.110570e+05,...,3.110570e+05,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000,311057.000000
mean,0.000554,55.835936,4.423247,0.773149,120.922991,0.513570,52.416291,51.557146,-44.155006,6.894767e+06,...,1.805554e+03,120.668965,3.021043,24.352143,12.558740,17.242533,-0.087675,120.836862,8.196109,0.355432
std,0.023981,28.097383,21.359055,11.522514,171.170431,7.714450,12.440937,10.112696,30.148964,2.222884e+08,...,1.047894e+06,170.668644,4.749986,9.942685,60.328666,112.381748,5.364749,170.869031,81.307466,4.487297
min,-0.565504,0.209197,-72.950037,-445.373728,3.699683,-82.601881,8.296877,12.395347,-100.000000,-6.003341e+09,...,-8.543236e+07,3.524507,0.133810,4.984696,-96.000000,-666.666667,-197.421130,3.782429,-772.579211,-176.718366
25%,-0.008284,30.703139,-9.917731,-1.531177,48.631518,-2.791966,44.086917,44.687654,-70.606777,-7.422926e+06,...,-2.445819e+04,48.522630,1.074057,16.807743,-44.000000,-67.805591,-1.083997,48.629439,-27.632909,-0.587163
50%,0.001068,59.515224,4.785451,0.582735,81.657529,0.898623,52.759183,51.710190,-40.361569,1.024160e+07,...,2.283651e+03,81.510848,1.800955,22.395021,32.000000,30.890230,-0.044998,81.649030,13.993008,0.242179
75%,0.009939,81.693311,19.364444,3.007058,137.473098,4.206078,61.213795,58.562741,-16.655445,3.541808e+07,...,3.104698e+04,137.243573,3.236269,30.004595,68.000000,102.557234,0.961001,137.430524,50.865529,1.212038
max,0.679796,99.762737,79.682334,430.632004,3075.141674,110.945263,92.939811,87.816558,-0.000000,1.755430e+09,...,2.120924e+08,3112.725203,116.803294,76.809178,96.000000,666.343730,193.065051,3118.971991,909.831225,162.689479
